# 모듈 불러오기

#### 기본

In [ ]:
import numpy as np
import pandas as pd
import joblib

In [ ]:
# !conda install numpy 
# !conda install pandas
# !conda install scikit-learn
# !conda install scipy
# !conda install tensorflow
# !conda install matplotlib
# !conda install seaborn

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install hyperopt
# !pip install -U imbalanced-learn
# !pip install missingno

#### 전처리

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer

#### 리샘플링

In [ ]:
from imblearn.over_sampling import (
    RandomOverSampler, 
    ADASYN, 
    SMOTE
)
from imblearn.under_sampling import (
    RandomUnderSampler, 
    TomekLinks, 
    CondensedNearestNeighbour, 
    OneSidedSelection, 
    EditedNearestNeighbours, 
    NeighbourhoodCleaningRule
)

#### 분석

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_ind, f_oneway, pearsonr, chi2_gen

#### 회귀

In [ ]:
from sklearn.linear_model import LinearRegression as RL
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR

from lightgbm import plot_importance as lgbm_plot_importance
from xgboost import plot_importance as xgb_plot_importance
from catboost import Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

#### 분류

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

#### 교차검증

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import cv
import hyperopt

from sklearn.model_selection import (
    StratifiedKFold, # 분류
    KFold, # 회귀
    # GroupKFold, 
    # RepeatedKFold, 
    # StratifiedGroupKFold, 
    # RepeatedStratifiedKFold
)

#### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 데이터 로드

In [ ]:
folder_name = 'data'
file_name = 'Feature_Website_fix'

df = pd.read_csv(f'./{folder_name}/{file_name}.csv', sep=',', encoding='utf-8')

# 데이터 분석 (단변량, 이변량, ydata-profiling)

#### 간단한 단변량 이변량 분석

In [ ]:
# !pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
profile = df.profile_report(interactions={'targets':['labels'],}, explorative=True)
profile.to_file("your_report.html")

#### 수작업으로 분석

In [ ]:
# 숫자형 -> 숫자형 이변량 분석
def numerical_analysis(df, x, y):
    plt.title('숫자형 -> 숫자형 이변량 분석')
    plt.grid(axis="x")
    
    sns.regplot(x = x, y = y, data = df)
    sns.jointplot(x = x, y = y, data = df)
    plt.show()    

    result = pearsonr(x = df[x], y = df[y])
    print(f'상관계수 : {result[0]}, p-value : {result[1]}')
    print('-' * 50)

# 범주형 -> 숫자형 이변량 분석
def categorical_analysis(df, x, y):
    plt.title('범주형 -> 숫자형 이변량 분석')
    plt.grid(axis="x")
    
    sns.barplot(x = x, y = y, data = df)
    plt.axhline(df[y].mean(), color = 'r')
    plt.show()    
    
    data_group = []
    cnt = 0
    
    for val in df[x].unique():
        data_group.append(df.loc[df[x]==val, y])
        cnt += 1

    if cnt == 2:
        result = ttest_ind(*data_group)
        print(f't-value : {result[0]}, p-value : {result[1]}')
        print('-' * 80)
    else:
        result = f_oneway(*data_group)
        print(f'f-value : {result[0]}, p-value : {result[1]}')
        print('-' * 80)

# 숫자형 -> 숫자형, 범주형 상관계수, t,f,p value 출력
def print_relation_xy(df, numerical_cols, categorical_cols, y):
    for x in categorical_cols:
        data_group = []
        cnt = 0
        
        for val in df[x].unique():
            data_group.append(df.loc[df[x]==val, y])
            cnt += 1
        
        if x == y: # target y는 출력 제외
            continue
        
        if cnt == 2:
            result = ttest_ind(*data_group)
            print(f'{x} t-value : {result[0]}, p-value : {result[1]}')
        else:
            result = f_oneway(*data_group)
            print(f'{x} f-value : {result[0]}, p-value : {result[1]}')

    print('-' * 80)
    
    for x in numerical_cols:
        result = pearsonr(x = df[x], y = df[y])
        print(f'{x} 상관계수 : {result[0]}, p-value : {result[1]}')

# 상관계수 히트맵 보기
def show_corr(df):
    plt.figure(figsize=(15,12))
    sns.heatmap(df.corr(), annot=True, cmap='RdYlGn_r')
    plt.xticks(rotation=45)
    plt.show()

# 범주형 -> 범주형 이변량 분석